<a href="https://colab.research.google.com/github/Abril1030/AbrilPublicRepo/blob/main/Bloque_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
Archivo = 'PRESION_NO_ALIN_V2.xlsx' #Carga el excel
Diccionario = pd.read_excel(Archivo, sheet_name=None, index_col=False) #Guarda todos los registros en un diccionario :) Los nombres de los libros están entre ''

In [ ]:
Sujetos = Diccionario.get('SUJETOS') #Separa los elementos del diccionario y los guarda en un dataframe :3
Sujetos #Ejemplo para mostrar como se ve el output

In [ ]:
Alineacion1 = Diccionario.get('ALIN1')
Alineacion2 = Diccionario.get('ALIN2')
Alineacion4 = Diccionario.get('ALIN4')
F21 = Diccionario.get('F21')
F22 = Diccionario.get('F22')
F24 = Diccionario.get('F24')
F61 = Diccionario.get('F61')
F62 = Diccionario.get('F62')
F64 = Diccionario.get('F64')
E21 = Diccionario.get('E21')
E22 = Diccionario.get('E22')
E24 = Diccionario.get('E24')
E61 = Diccionario.get('E61')
E62 = Diccionario.get('E62')
E64 = Diccionario.get('E64')
AB21 = Diccionario.get('AB21')
AB22 = Diccionario.get('AB22')
AB24 = Diccionario.get('AB24')
AB61 = Diccionario.get('AB61')
AB62 = Diccionario.get('AB62')
AB64 = Diccionario.get('AB64')
AD21 = Diccionario.get('AD21')
AD22 = Diccionario.get('AD22')
AD24 = Diccionario.get('AD24')
AD61 = Diccionario.get('AD61')
AD62 = Diccionario.get('AD62')
AD64 = Diccionario.get('AD64')

In [ ]:
Caracteristicas1 = Alineacion1.iloc[:7,:3] #Para llamar las características o datos del sujeto en cada libro, revisar si son necesarias. Si no, usar drop hasta el rango de filas útil
Caracteristicas1

,file name: RODRIGUEZ_HENRRY_2.sol,date/time 16.08.14 15.52,Unnamed: 2
0,sensor type: we-519l-520r-30-04,NaN,NaN
1,total time [secs]: 11.700,time per frame [secs]: 0.02000,scanning rate [Hz]: 50
2,pressure values in kPa,NaN,NaN
3,NaN,NaN,NaN
4,mask definition:,NaN,NaN
5,"left insole: 1,2,3,4,5,6,7,8,9,10,11,12,13,14,...",NaN,NaN
6,"right insole: 1,2,3,4,5,6,7,8,9,10,11,12,13,14...",NaN,NaN


In [ ]:
LecturasALIN1 = Alineacion1.iloc[8:596,:199] #Para llamar las lecturas de los sensores.
EncabezadoLecturasALIN1 = LecturasALIN1.iloc[0]
LecturasALIN1.columns = EncabezadoLecturasALIN1
LecturasALIN1=LecturasALIN1.drop(labels=[8], axis=0)
LecturasALIN1

8,time[secs],1,2,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0
9,0.02000,0.000,0.000,0.0,0.0,0.0,0.0,45000.0,25000.0,20000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.04000,0.000,15000,0.0,0.0,0.0,0.0,47500.0,25000.0,20000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.06000,0.000,15000,15000.0,0.0,0.0,0.0,45000.0,25000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.08000,0.000,15000,0.0,0.0,0.0,0.0,45000.0,25000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.10000,0.000,15000,15000.0,0.0,0.0,0.0,45000.0,25000.0,20000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,1162000,0.000,0.000,0.0,0.0,0.0,0.0,42500.0,20000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
590,1164000,0.000,0.000,0.0,0.0,0.0,0.0,42500.0,20000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
591,1166000,0.000,0.000,0.0,0.0,0.0,0.0,40000.0,20000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
592,1168000,0.000,0.000,0.0,0.0,0.0,0.0,42500.0,20000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import codecs
import cairosvg
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.animation as animation
#from numba import njit
from io import BytesIO
from PIL import Image

In [ ]:
def png_bytes_to_numpy(png):
    """Convert png bytes to numpy array

    Example:  

    >>> fig = go.Figure(go.Scatter(x=[1], y=[1]))
    >>> plt.imshow(png_bytes_to_numpy(fig.to_image('png')))
    """
    return np.array(Image.open(BytesIO(png)))

In [ ]:
"""funcion que se encarga de formatear la informacion para lo que se requiere"""
def get_form_data(Diccionary,sheet):
    alineacion = Diccionary.get('ALIN4')
    alingREad = alineacion.iloc[8:596,:199] #Para llamar las lecturas de los sensores.
    EncabezadoAling = alingREad.iloc[0]
    alingREad.columns = EncabezadoAling
    alingREad=alingREad.drop(labels=[8], axis=0)
    return alingREad

In [ ]:
def init_fnc(filename):
    global content_svg
    content_svg=[]
    for _file in filename:
        with codecs.open(_file, encoding='utf-8', errors='ignore') as f:
            svg_as_text=f.read()
            svg_as_text=svg_as_text.replace('"st0"', '"stk"')
            for i in range(99):
                svg_as_text=svg_as_text.replace('"stk"','"st{}"'.format(i),1)
            content_svg.append(svg_as_text)

In [ ]:
def svg2img(foot,data_sensors):
    global content_svg
    
    content=content_svg[foot]
    style_m1=".st{0}{1}\n\t"
    style_m2="{{fill:#{0}}}"
    style_temp=""
    
    for i in range(len(data_sensors)):
        k=''
        k=['0' for i in range(6-len(data_sensors[i]))]
        k="".join(k)
        if len(data_sensors[i])>0:
            style_temp+=style_m1.format(i,style_m2.format(k+str(data_sensors[i])+';'))
        else:
            style_temp+=style_m1.format(i,style_m2.format('000000;'))

    style_c=content.find('.st0')
    sub_str=content[style_c:style_c+19]
    svg_loc=content.replace(sub_str, style_temp)

    png_data = cairosvg.svg2png(bytestring=svg_loc.encode())
    img_out=png_bytes_to_numpy(png_data)
    return img_out

In [ ]:
Archivo = 'PRESION_NO_ALIN_V2.xlsx' #Carga el excel
Diccionario = pd.read_excel(Archivo, sheet_name=None, index_col=False)

#Sujetos = Diccionario.get('SUJETOS')

init_fnc(["foot L.svg","foot R.svg"])#Precarga los svg del directorio para L y R


#definiciones para animación 

fig, (ax1, ax2) = plt.subplots(1,2)
x=0
ims = []

data_readed= get_form_data(Diccionario,'ALIN1')
max_dts=data_readed.max(skipna=True)
max_dt=max(max_dts)


for i in range(len(data_readed)-10):
    heat_map = data_readed.iloc[i].to_numpy(copy=True)
    heat_map=list(map(float, heat_map))
    heat_map=[(y/max_dt)*0x00FFFF for y in heat_map]
    heat_map_v=list(map(int, heat_map))

    data_sensor_hex=[hex(data)for data in heat_map_v]
    data_sensor_str=[data_sh.replace('0x','') for data_sh in data_sensor_hex]
    Img_L=svg2img(0,data_sensor_str[0:99])#0 para izquierda
    Img_R=svg2img(1,data_sensor_str[1:199])#1 para derecha

    im_l= ax1.imshow(Img_L, animated=True)
    im_r= ax2.imshow(Img_R, animated=True)
    if i == 0:
        ax1.imshow(Img_L)  # show an initial one first
        ax2.imshow(Img_R)
    ims.append([im_l,im_r])

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)
plt.show() #Funciona